In [2]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Week4.0-Machine Learning-Classification/CLassification/2.Grid Search/Social_Network_Ads.csv")

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [7]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [8]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']}



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)




Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [14]:
# print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [15]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'auto', 'splitter': 'best'}: 0.9104477611940298


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [ 6 43]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93        85
           1       0.88      0.88      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [18]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.9034813925570228

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003562,0.001039,0.003104,0.000945,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",0.826263,0.832918,0.832483,0.925272,0.905069,0.864401,0.042009,1
1,0.003233,0.000121,0.002598,0.000136,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",0.799620,0.850809,0.791152,0.851527,0.832483,0.825118,0.025360,11
2,0.002961,0.002496,0.003151,0.001084,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.804764,0.831253,0.833323,0.886792,0.885265,0.848279,0.032430,5
3,0.001869,0.000627,0.003278,0.001161,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.808927,0.888107,0.795256,0.850543,0.813179,0.831202,0.033839,9
4,0.001576,0.000069,0.003539,0.001785,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.799620,0.870898,0.813179,0.832483,0.903610,0.843958,0.038286,6
5,0.001542,0.000064,0.002669,0.000067,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.832255,0.866968,0.813179,0.867097,0.795256,0.834951,0.028689,8
6,0.002630,0.000683,0.004533,0.003049,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",0.829615,0.850809,0.832483,0.870362,0.905069,0.857668,0.027844,3
7,0.003340,0.000606,0.003246,0.000578,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",0.826263,0.813511,0.791152,0.851527,0.865054,0.829502,0.026405,10
8,0.001699,0.000210,0.002685,0.000048,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.776290,0.868752,0.832483,0.832483,0.883278,0.838657,0.037047,7
9,0.001828,0.000698,0.002678,0.000062,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.829615,0.850809,0.850543,0.905069,0.850543,0.857316,0.025226,4


In [22]:
age_input=float(input("Age:"))
EstimatedSalary_input=float(input("EstimatedSalary':"))
Gender_Male_input=int(input("Sex Male 0 or 1:"))


Age:30
EstimatedSalary':10000000
Sex Male 0 or 1:1


In [23]:
Future_Prediction=grid.predict([[age_input,EstimatedSalary_input,Gender_Male_input  ]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
